In [1]:
import warnings
import requests
import os
import glob
import pandas as pd
import openbabel
import numpy as np
from plip.structure.preparation import PDBComplex
from plip.exchange.report import BindingSiteReport
from rdkit import Chem
from rdkit.Chem import AllChem
from biopandas.pdb import PandasPdb
from Bio.PDB.SASA import ShrakeRupley
from Bio.PDB import PDBParser
import oddt
from oddt import fingerprints
warnings.filterwarnings("ignore")

class PLIF:
    def __init__(self, PDB: str, MOL_SPLIT_START: int = 70, **kwargs):
        kwargs.setdefault('aggr', 'add')
        super(PLIF,self).__init__()
        
        self.MOL_SPLIT_START=MOL_SPLIT_START
        self.pdb=PDB
        self.records=['ATOM']
        self.values=['HOH','CL','MG','ZN','MN','CA']
        self.interaction_slices={"hydrophobic":[0,1,6,7,8,9,10],
            "hbond":[0,1,7,11,13,15,16],
            "waterbridge":[0,1,[6,7],11,13,16,17],
            "saltbridge":[0,1,7,10,3,11,12],
            "pistacking":[0,1,7,11,6,12,13],
            "pication":[0,1,7,11,3,12,13],
            "halogen":[0,1,7,10,12,14,15],
            "metal":[0,1,11,8,6,17,16]} 

        self.column_names = ['RESNR', 'RESTYPE', 'DIST', 'LIG_IDX','PROT_IDX','FRAGMENT_ATOMS_COORDS', 'AA_COORDS']
        self.path = os.getcwd()


    def okToBreak(self, bond):
        """
        Here we apply a bunch of rules to judge if the bond is OK to break.

        Parameters
        ----------
        bond :
            RDkit MOL object

        Returns
        -------
        Boolean :
            OK or not to break.
        """
        # See if the bond is in Ring (don't break that)
        if bond.IsInRing():
            return False
        # We OK only single bonds to break
        if bond.GetBondType() != Chem.rdchem.BondType.SINGLE:
            return False

        # Get the beginning atom of the bond
        begin_atom = bond.GetBeginAtom()
        # Get the ending atom of the bond
        end_atom = bond.GetEndAtom()
        # What kind of neighbors does these end and begenning atoms have? We need a family of no less than 5!
        neighbor_end=list(end_atom.GetNeighbors())
        neighbor_begin=list(begin_atom.GetNeighbors())
        if (len(neighbor_end) + len(neighbor_begin)) <5:
            return False
        #for atm in neighbor_end:
            #print(atm.GetAtomicNum())
        #print(begin_atom.GetAtomicNum(), end_atom.GetAtomicNum(), MOL_SPLIT_START)
        
        # Now check if end or begenning atoms are in ring (we dont wanna bother those)
        if not(begin_atom.IsInRing() or end_atom.IsInRing()):
            return False
        elif begin_atom.GetAtomicNum() >= self.MOL_SPLIT_START or \
                end_atom.GetAtomicNum() >= self.MOL_SPLIT_START:
            return False
        elif end_atom.GetAtomicNum() == 1:
            return False
        else:
            return True

    def undo_id_label (self, frag, split_id):
        # I am trying to restore Hydrogens where the break happened
        for i, atom in enumerate(frag.GetAtoms()):
            if atom.GetAtomicNum() >= split_id:
                atom.SetAtomicNum(1)

        return frag

    # Divide a molecule into fragments
    def split_molecule(self, mol, pdb):

        split_id = self.MOL_SPLIT_START

        res = []
        res_no_id=[]

        to_check = [mol]
        while len(to_check) > 0:
            ms = self.spf(to_check.pop(), split_id)
            if len(ms) == 1:
                res += ms
            else:
                to_check += ms
                split_id += 1
        for frag in res:
            res_no_id.append(self.undo_id_label(frag, self.MOL_SPLIT_START))

        res_pdb_frags=[]

        for idx, frag in enumerate(res_no_id):
            w = Chem.PDBWriter(f"tmp_{pdb}_{self.MOL_SPLIT_START+idx}.pdb")
            w.write(frag)
            w.close()
            
            unwanted_entries= ['CONECT', 'END']            
            with open(f"tmp_{pdb}_{self.MOL_SPLIT_START+idx}.pdb") as oldfile, open(f"{pdb}_{self.MOL_SPLIT_START+idx}.pdb", 'w') as newfile:
                for line in oldfile:
                    if not any(unwanted_entry in line for unwanted_entry in unwanted_entries):
                        newfile.write(line)

                    
            data = data2 = ""

            # Reading data from file1
            with open(f"ATOM_{pdb}.pdb") as fp:
                data = fp.read()

            # Reading data from file2

            with open(f"{pdb}_{self.MOL_SPLIT_START+idx}.pdb") as fp:
                data2 = fp.read()
            
            # Merging 2 files
            # To add the data of file2
            # from next line
            #data += "\n"
            data += data2
            
            with open(f"HOH_{pdb}.pdb") as fp:
                data3 = fp.read()
            data += data3

            with open (f"ATOM_{pdb}_{self.MOL_SPLIT_START+idx}.pdb", 'w') as fp:
                fp.write(data)
            res_pdb_frags.append(f"ATOM_{pdb}_{self.MOL_SPLIT_START+idx}.pdb")
        return res_pdb_frags #create_chain(res)


    # Function for doing all the nitty gritty splitting work.
    # loops over bonds until bonds get exhausted or bonds are ok to break, whichever comes first. If ok to break, then each
    # fragment needs to be checked individually again through the loop
    def spf(self, mol, split_id):

        bonds = mol.GetBonds()
        for i in range(len(bonds)):
            if self.okToBreak(bonds[i]):
                mol = Chem.FragmentOnBonds(mol, [i])
                # Dummy atoms are always added last
                n_at = mol.GetNumAtoms()
                print('Split ID', split_id)
                mol.GetAtomWithIdx(n_at-1).SetAtomicNum(split_id)
                mol.GetAtomWithIdx(n_at-2).SetAtomicNum(split_id)
                return Chem.rdmolops.GetMolFrags(mol, asMols=True)

        # If the molecule could not been split, return original molecule
        return [mol]
    #get_fragments(fragment_mols)

    def retreive_plip_interactions(self, pdb_file):
        """
        Retreives the interactions from PLIP.

        Parameters
        ----------
        pdb_file :
            The PDB file of the complex. 

        Returns
        -------
        dict :
            A dictionary of the binding sites and the interactions.
        """
        protlig = PDBComplex()   #instantiate the loader from PLIP
        protlig.load_pdb(pdb_file)   # load the pdb file
        for ligand in protlig.ligands:
            protlig.characterize_complex(ligand)   # find ligands and analyze interactions
        sites = {}
        # loop over binding sites
        for key, site in sorted(protlig.interaction_sets.items()):
            binding_site = BindingSiteReport(site)   # collect data about interactions
            # tuples of *_features and *_info will be converted to pandas DataFrame
            keys = (
                "hydrophobic",
                "hbond",
                "waterbridge",
                "saltbridge",
                "pistacking",
                "pication",
                "halogen",
                "metal"
            )
        # interactions is a dictionary which contains relevant information for each
        # of the possible interactions: hydrophobic, hbond, etc. in the considered
        # binding site. Each interaction contains a list with 
        # 1. the features of that interaction, e.g. for hydrophobic:
        # ('RES_number', 'RES_type', ..., 'LIG_coord', 'PROT_coord')
        # 2. information for each of these features, e.g. for hydrophobic
        # ('RES_number', 'RES_type', ..., 'LIG_coord', 'PROT_coord')

            interactions = {
                k: [getattr(binding_site, k + "_features")] + getattr(binding_site, k + "_info")
                for k in keys
            }
            sites[key] = interactions
        return sites

    def get_coords_prot(self, RESNR):
        ppdb = PandasPdb()
        ppdb.read_pdb(f"{self.pdb.split('.')[0]}_protein.pdb")
        only_protein=ppdb.df['ATOM']
        resnr_coords=[]
        for i in RESNR:
            resnr_coords.append(list(only_protein[only_protein['atom_number']==int(i)][['x_coord', 'y_coord', 'z_coord']].values[0]))
        return resnr_coords
    
    ### the most slow function out of all this garbage code ###
    def aa_descriptors_asa(self, fragment_idx, aa, coords, extra_feats):
        p = PDBParser(QUIET=1)
        structure = p.get_structure(self.pdb.split('.')[0], f"ATOM_{self.pdb.split('.')[0]}_{fragment_idx}.pdb")
        sr = ShrakeRupley()
        
        sasa_res=[]
        sasa_atom=[]

        for a in aa: 
            for chain in structure[0]:
                for res in chain:
                    if f'={a} ' in str(res.__repr__()):
                        sr.compute(structure[0], level="R")
                        sasa_res.append(round(res.sasa,2))
                        sr.compute(structure[0], level="A")
                        for coor in coords:
                            for atom in res:
                                if [round(item) for item in atom.get_coord()] == [round(np.float32(item)) for item in coor]:
                                    sasa_atom.append(round(atom.sasa,2))

#                         #if all(v == 0.0 for v in sasas):
                        if not sasa_atom:
                            try:
                                coords=self.get_coords_prot(extra_feats[0].split(',') if ',' in extra_feats[0] \
                                                               else extra_feats)
                                for coor in coords:
                                    for atom in res:
                                        if [round(item) for item in atom.get_coord()] == [round(np.float32(item)) for item in coor]:
                                            sasa_atom.append(round(atom.sasa,2))
                            except Exception:
                                print(f"no SASA for AA idx {a}'s atom'")
                                
                
        return [sasa_atom], [sasa_res] , coords
                
    def interaction_df(self, split):

        all_interactions_df = pd.DataFrame()


        # We create the dictionary for the complex of interest:
        for idx, s in enumerate(split):

            pdb_id=s.split('.')[0]
            raw=pdb_id.split('_')[1]
            idx_frag=int(pdb_id.split('_')[2])
            interactions_by_site = self.retreive_plip_interactions(f"{pdb_id}.pdb")

            # Let’s see how many binding sites are detected:

    #         print(
    #             f"Number of binding sites detected in {pdb_id} : "
    #             f"{len(interactions_by_site)}\n"
    #             f"with {interactions_by_site.keys()}"
    #         )
            # In this case, the first binding site containing ligand 03P will be further investigated.
            index_of_selected_site = 0
            selected_site = list(interactions_by_site.keys())[index_of_selected_site]
            #print(selected_site)


            valid_types = [
                    "hydrophobic",
                    "hbond",
                    "waterbridge",
                    "saltbridge",
                    "pistacking",
                    "pication",
                    "halogen",
                    "metal",
                ]

            for _type in valid_types:
                output_df=self.create_df_from_binding_site(raw, interactions_by_site[selected_site], idx+self.MOL_SPLIT_START, selected_site,
                                                      interactions_by_site,
                                                      interaction_type=_type)
                all_interactions_df=all_interactions_df.append(output_df)
        all_interactions_df = all_interactions_df[all_interactions_df['RESNR'].notna()]
        all_interactions_df.to_csv(f"{self.path}/results_plifs/{raw}_plifs_and_properties.csv", index=False)
        return all_interactions_df


    # We can construct a pandas.DataFrame for a binding site and particular interaction type.

    def create_df_from_binding_site(self, raw, selected_site_interactions, fragment_idx, selected_site, 
                                    interactions_by_site, interaction_type="hbond"):
        """
        Creates a data frame from a binding site and interaction type.

        Parameters
        ----------
        selected_site_interactions : dict
            Precalculated interactions from PLIP for the selected site
        interaction_type : str
            The interaction type of interest (default set to hydrogen bonding).

        Returns
        -------
        pd.DataFrame :
            DataFrame with information retreived from PLIP.
        """
        # check if interaction type is valid:
        valid_types = [
            "hydrophobic",
            "hbond",
            "waterbridge",
            "saltbridge",
            "pistacking",
            "pication",
            "halogen",
            "metal",
        ]


        if interaction_type not in valid_types:
            print("!!! Wrong interaction type specified. Hbond is chosen by default !!! \n")
            interaction_type = "hbond"

        def interaction_values(n):
            try:
                interactions=interactions_by_site[selected_site][interaction_type]
                if type(n) is list:
                    return [interactions[1:][x][i] for x in 
                        range(len(interactions[1:])) for i in n]
                else:
                    return [interactions[1:][x][n] for x in 
                        range(len(interactions[1:]))]
            except Exception:
                return None
            
        if interactions_by_site[selected_site][interaction_type][1:]:
            #print(list(map(interaction_values, self.interaction_slices[interaction_type])), self.column_names)
            selected_feats=list(map(interaction_values, self.interaction_slices[interaction_type]))
            #print(selected_feats)
            try: 
                if int(selected_feats[4])>int(selected_feats[3]):
                    selected_feats[3], selected_feats[4] = selected_feats[4], selected_feats[3]  
            except: 
                if int(any(selected_feats[4]))>int(any(selected_feats[3])):
                    selected_feats[3], selected_feats[4] = selected_feats[4], selected_feats[3] 
            df = pd.DataFrame(
                # data is stored AFTER the columns names
                [selected_feats],
                # column names are always the first element - we skipped that in the above - we are gonna use that for naming the df
                columns = self.column_names
            )

            df["INTERACTION_TYPE"]=interaction_type
            df["ATOM_SASA"], df["AA_SASA"], extra_coords = self.aa_descriptors_asa(fragment_idx,selected_feats[0],
                                                                  [list(x) for x in selected_feats[6]],
                                                                 selected_feats[4]) 
                                                                  #df["AA_COORDS"].values[0])
            df["AA_COORDS"]=[extra_coords]
                #[self.get_coords_prot(selected_feats[4].split(','))]
            df["FRAGMENT_ATOMS_COORDS"]=[selected_feats[5]]
                            #[self.get_coords_lig(selected_feats[3].split(','))]    
            df['FRAGMENT_ID']=fragment_idx

            # ideally we would like to exclude waters from further processing. Threrfore let us reduce any waterbridge 
            # interaction to the eucladean distance in order to omit water
            
            if interaction_type == "waterbridge":
                df['DIST']=[[np.linalg.norm(x) for x in df['DIST'].to_numpy()]]
                
            # also deal with one distance value and two coords, this is common in saltbridge interactions:
            if len(extra_coords) == len(selected_feats[2])*2:
                df['DIST']=[selected_feats[2] + selected_feats[2]]
                
        else:

            df= pd.DataFrame({'RESNR':[None], 'RESTYPE':[None], 'DIST':[None], 'LIG_IDX':[None],'PROT_IDX':[None],
                        'INTERACTION_TYPE':[interaction_type], "AA_COORDS": [None], "FRAGMENT_ATOMS_COORDS":[None],
                        "ATOM_SASA":[None],"AA_SASA":[None],
                              'FRAGMENT_ID':[str(fragment_idx)]})



        return df
    def pdb_2_sdf(self, pdb):
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats("pdb", "sdf")
        mol = openbabel.OBMol()
        obConversion.ReadFile(mol, pdb)   # Open Babel will uncompress automatically

        mol.AddHydrogens()


        obConversion.WriteFile(mol, f"{pdb.split('.')[0]}.sdf")
        return f"{pdb.split('.')[0]}.sdf"
    
    def sdf_2_pdb(self, sdf):
        obConversion = openbabel.OBConversion()
        obConversion.SetInAndOutFormats("sdf", "pdb")
        mol = openbabel.OBMol()
        obConversion.ReadFile(mol, sdf)   # Open Babel will uncompress automatically

        mol.AddHydrogens()
        obConversion.WriteFile(mol, f"{sdf.split('.')[0]}.pdb")
        return f"HETATM_{sdf.split('.')[0]}.pdb"

    def save_bpdb(self, pdb,ppdb, record):  
        ppdb.to_pdb(path=f"{record}_{pdb.split('.')[0].split('_')[0]}.pdb",
                    records=[record],
                    gz=False, 
                    append_newline=True)

    def get_HOH_pdb(self, pdb):
        ppdb = PandasPdb() 
        ppdb.read_pdb(pdb) 
        ppdb.df['HETATM']=ppdb.df['HETATM'].loc[ppdb.df['HETATM']['residue_name'].isin(self.values)]
        ppdb.to_pdb(path=f"HOH_{pdb.split('.')[0].split('_')[0]}.pdb",
                records=['HETATM'],
                gz=False, 
                append_newline=True)

    def keep_relevant_hetatm(self, pdb):
#         pdb = self.pdb
        ppdb = PandasPdb() 
        ppdb.read_pdb(pdb)    

        ##ppdb.df['HETATM']=ppdb.df['HETATM'][ppdb.df['HETATM'].residue_name == self.getLigandName(str(pdb).split('.')[0])]  
        for c in self.records:
            self.save_bpdb(pdb,ppdb, c)
        self.get_HOH_pdb(pdb)
        return
    
    
    def fragment_and_plif(self):
        path = os.getcwd()
        if not os.path.exists('results_plifs'):
            os.mkdir(f'{path}/results_plifs')
         
        raw=str(self.pdb).split('.')[0]
        self.keep_relevant_hetatm(f'{raw}_protein.pdb')
        self.sdf_2_pdb(f'{raw}_ligand.sdf')
        fragment_mols = Chem.SDMolSupplier(str(f'{raw}_ligand.sdf'), removeHs=True, sanitize=False)
        try: fragment_mols = Chem.RemoveHs(fragment_mols[0])
        except: fragment_mols = AllChem.MolFromPDBFile(f'{raw}_ligand.pdb')
        output_df = self.interaction_df(self.split_molecule(fragment_mols,raw))
        fileList = []
        fileList.extend(glob.glob(f'{path}/{raw}_7*pdb'))
        fileList.extend(glob.glob(f'{path}/{raw}_8*pdb'))
        fileList.extend(glob.glob(f'{path}/{raw}_9*pdb'))
        fileList.extend(glob.glob(f'{path}/*_{raw}*pdb'))
        fileList.extend(glob.glob(f'{path}/*_{raw}*sdf'))
#         for filePath in fileList:
#             try:
#                 os.remove(filePath)
#             except:
#                 print("Error while deleting file : ", filePath)
#         os.chdir(f'{path}')
        
        return output_df.groupby('FRAGMENT_ID')['AA_COORDS', 'FRAGMENT_ATOMS_COORDS','INTERACTION_TYPE','DIST','ATOM_SASA','AA_SASA'].agg(list)
    
    
if __name__ == "__main__":
    
    df = PLIF(PDB = '5te0.pdb').fragment_and_plif()
    

ModuleNotFoundError: No module named 'plip'